# 🌍 International Stock Index Analysis V4
## Institutional-Grade Analysis

**Author:** Created for Ferhat Culfaz  
**Date:** January 2025

---

### 📊 Features Included:

| Feature | Description |
|---------|-------------|
| **Options-Implied Volatility** | VIX term structure, IV-RV spread, volatility risk premium |
| **Correlation Regimes** | Correlation matrices by VIX regime (Low/Normal/High) |
| **Factor Decomposition** | Market, Value, Momentum, Quality, Size, Low Vol factors |
| **Monte Carlo Simulations** | 10,000 simulations, VaR, CVaR, stress testing |
| **60+ Geopolitical Events** | Trump tariffs, Israel-Iran, Ukraine-Russia, semiconductors |
| **17 International Indices** | US, Europe, Asia, Oceania |

---

### 🚀 Platform Compatibility:
- ✅ Kaggle
- ✅ Google Colab
- ✅ Local Jupyter
- ✅ JupyterLab
- ✅ VS Code Notebooks

---
## 1️⃣ Setup & Installation

Run this cell first to install required packages (if not already installed).

In [1]:
# Install packages (uncomment if needed)
# !pip install yfinance pandas numpy plotly scipy statsmodels seaborn -q

# For Kaggle/Colab - these are usually pre-installed
import sys
print(f"Python version: {sys.version}")

Python version: 3.14.2 (tags/v3.14.2:df79316, Dec  5 2025, 17:18:21) [MSC v.1944 64 bit (AMD64)]


---
## 2️⃣ Import Libraries

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy import stats
import warnings
import os

# Optional imports
try:
    from statsmodels.regression.linear_model import OLS
    from statsmodels.tools import add_constant
    STATSMODELS_AVAILABLE = True
    print("✅ statsmodels available")
except ImportError:
    STATSMODELS_AVAILABLE = False
    print("⚠️ statsmodels not available - factor analysis limited")

try:
    import seaborn as sns
    import matplotlib.pyplot as plt
    SEABORN_AVAILABLE = True
    print("✅ seaborn/matplotlib available")
except ImportError:
    SEABORN_AVAILABLE = False

warnings.filterwarnings('ignore')

# For Plotly in notebooks
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

print("\n✅ All libraries imported successfully!")

✅ statsmodels available
✅ seaborn/matplotlib available



✅ All libraries imported successfully!


---
## 3️⃣ Configuration

Define all tickers, regions, and parameters.

In [3]:
# ============================================================================
# INDEX TICKERS (17 Countries)
# ============================================================================
INDEX_TICKERS = {
    'SPY': 'SPY',           # US S&P 500
    'France': '^FCHI',      # CAC 40
    'Germany': '^GDAXI',    # DAX
    'UK': '^FTSE',          # FTSE 100
    'Italy': 'FTSEMIB.MI',  # FTSE MIB
    'Sweden': '^OMX',       # OMX Stockholm 30
    'Spain': '^IBEX',       # IBEX 35
    'Norway': '^OBX',       # OBX Index
    'Denmark': '^OMXC25',   # OMX Copenhagen 25
    'Switzerland': '^SSMI', # SMI
    'Finland': '^OMXH25',   # OMX Helsinki 25
    'Korea': '^KS11',       # KOSPI
    'Japan': '^N225',       # Nikkei 225
    'China': '000001.SS',   # Shanghai Composite
    'Hong Kong': '^HSI',    # Hang Seng
    'Canada': '^GSPTSE',    # TSX Composite
    'Australia': '^AXJO',   # ASX 200
}

# ============================================================================
# FACTOR ETFs
# ============================================================================
FACTOR_TICKERS = {
    'Market': 'SPY',        # Market factor
    'Value': 'IVE',         # S&P 500 Value
    'Growth': 'IVW',        # S&P 500 Growth
    'Momentum': 'MTUM',     # iShares Momentum
    'Quality': 'QUAL',      # iShares Quality
    'Size': 'IWM',          # Russell 2000 (Small Cap)
    'Low Vol': 'USMV',      # Min Volatility
}

# ============================================================================
# VOLATILITY INDICES
# ============================================================================
VOLATILITY_TICKERS = {
    'VIX': '^VIX',          # 30-day implied vol
    'VIX9D': '^VIX9D',      # 9-day implied vol
    'VIX3M': '^VIX3M',      # 3-month implied vol
    'VIX6M': '^VIX6M',      # 6-month implied vol
    'VVIX': '^VVIX',        # Vol of VIX
}

# ============================================================================
# SECTOR ETFs
# ============================================================================
SECTOR_TICKERS = {
    'US Tech': 'QQQ',
    'US Semiconductors': 'SOXX',
    'US Utilities': 'XLU',
    'US Healthcare': 'XLV',
    'Gold': 'GLD',
}

# ============================================================================
# ETF ALTERNATIVES (fallback if index fails)
# ============================================================================
ETF_ALTERNATIVES = {
    'France': 'EWQ', 'Germany': 'EWG', 'UK': 'EWU', 'Italy': 'EWI',
    'Sweden': 'EWD', 'Spain': 'EWP', 'Switzerland': 'EWL', 'Korea': 'EWY',
    'Japan': 'EWJ', 'China': 'MCHI', 'Hong Kong': 'EWH', 'Canada': 'EWC',
    'Australia': 'EWA',
}

# ============================================================================
# REGIONS
# ============================================================================
REGIONS = {
    'North America': ['SPY', 'Canada'],
    'Western Europe': ['Germany', 'France', 'UK', 'Switzerland'],
    'Northern Europe': ['Sweden', 'Norway', 'Denmark', 'Finland'],
    'Southern Europe': ['Italy', 'Spain'],
    'Asia Developed': ['Japan', 'Korea', 'Hong Kong'],
    'Asia Emerging': ['China'],
    'Oceania': ['Australia'],
}

COUNTRY_TO_REGION = {}
for region, countries in REGIONS.items():
    for country in countries:
        COUNTRY_TO_REGION[country] = region

print(f"✅ Configured {len(INDEX_TICKERS)} indices, {len(FACTOR_TICKERS)} factors, {len(VOLATILITY_TICKERS)} vol indices")

✅ Configured 17 indices, 7 factors, 5 vol indices


---
## 4️⃣ Geopolitical Events Database (60+ Events)

In [4]:
GEOPOLITICAL_EVENTS = {
    # Trump Administration
    '2024-11-05': {'event': 'Trump Election Victory', 'color': '#1f77b4', 'category': 'trump', 'severity': 'high'},
    '2024-12-17': {'event': 'Trump Tariff Threats Begin', 'color': '#ff7f0e', 'category': 'trump', 'severity': 'high'},
    '2025-01-20': {'event': 'Trump Inauguration', 'color': '#d62728', 'category': 'trump', 'severity': 'high'},
    '2025-01-23': {'event': 'Greenland/Denmark Tariff Threats', 'color': '#ff7f0e', 'category': 'trump', 'severity': 'medium'},
    '2025-02-01': {'event': 'USMCA Tariffs Effective', 'color': '#8c564b', 'category': 'trump', 'severity': 'high'},
    '2025-02-04': {'event': 'China Tariffs +10%', 'color': '#e377c2', 'category': 'trump', 'severity': 'high'},
    '2025-02-10': {'event': 'Steel/Aluminum Tariffs', 'color': '#7f7f7f', 'category': 'trump', 'severity': 'high'},
    '2025-02-13': {'event': 'Trump-Putin Phone Call', 'color': '#17becf', 'category': 'trump', 'severity': 'high'},
    '2025-03-04': {'event': 'Tariff Escalation Day', 'color': '#bcbd22', 'category': 'trump', 'severity': 'high'},
    '2025-03-12': {'event': 'EU Counter-Tariffs Announced', 'color': '#17becf', 'category': 'eu', 'severity': 'high'},
    '2025-04-02': {'event': '"Liberation Day" Tariffs', 'color': '#d62728', 'category': 'trump', 'severity': 'critical'},
    '2025-04-09': {'event': '90-Day Tariff Pause', 'color': '#2ca02c', 'category': 'trump', 'severity': 'critical'},
    '2025-05-12': {'event': 'US-China Geneva Talks', 'color': '#ff9896', 'category': 'trump', 'severity': 'high'},
    
    # UK Trade
    '2024-07-05': {'event': 'UK Labour Government', 'color': '#1f77b4', 'category': 'uk', 'severity': 'medium'},
    '2025-01-25': {'event': 'UK-Trump Trade Optimism', 'color': '#2ca02c', 'category': 'uk', 'severity': 'medium'},
    '2025-03-08': {'event': 'UK-US Trade Framework', 'color': '#2ca02c', 'category': 'uk', 'severity': 'high'},
    '2025-04-12': {'event': 'UK Tariff Exemption Granted', 'color': '#2ca02c', 'category': 'uk', 'severity': 'high'},
    
    # Semiconductors
    '2023-07-23': {'event': 'Japan Chip Export Controls', 'color': '#9467bd', 'category': 'semiconductors', 'severity': 'high'},
    '2023-10-17': {'event': 'US Expands China Chip Ban', 'color': '#9467bd', 'category': 'semiconductors', 'severity': 'high'},
    '2024-07-15': {'event': 'ASML Export Restrictions', 'color': '#9467bd', 'category': 'semiconductors', 'severity': 'high'},
    '2024-12-02': {'event': 'China Chip Retaliation', 'color': '#9467bd', 'category': 'semiconductors', 'severity': 'high'},
    '2025-03-18': {'event': 'Enhanced Chip Restrictions', 'color': '#9467bd', 'category': 'semiconductors', 'severity': 'high'},
    
    # Israel-Iran
    '2023-10-07': {'event': 'Hamas Attack on Israel', 'color': '#d62728', 'category': 'israel', 'severity': 'critical'},
    '2023-10-27': {'event': 'Israel Ground Invasion Gaza', 'color': '#ff7f0e', 'category': 'israel', 'severity': 'high'},
    '2024-04-13': {'event': 'Iran Drone Attack on Israel', 'color': '#d62728', 'category': 'israel', 'severity': 'critical'},
    '2024-10-01': {'event': 'Iran Ballistic Missile Attack', 'color': '#d62728', 'category': 'israel', 'severity': 'critical'},
    '2024-10-26': {'event': 'Israel Strikes Iran', 'color': '#ff7f0e', 'category': 'israel', 'severity': 'high'},
    '2025-01-15': {'event': 'Gaza Ceasefire Agreement', 'color': '#2ca02c', 'category': 'israel', 'severity': 'high'},
    
    # Ukraine-Russia
    '2023-06-06': {'event': 'Kakhovka Dam Destruction', 'color': '#d62728', 'category': 'ukraine', 'severity': 'high'},
    '2023-06-24': {'event': 'Wagner Mutiny', 'color': '#ff7f0e', 'category': 'ukraine', 'severity': 'high'},
    '2024-02-16': {'event': 'Navalny Death', 'color': '#d62728', 'category': 'ukraine', 'severity': 'high'},
    '2024-08-06': {'event': 'Ukraine Kursk Incursion', 'color': '#1f77b4', 'category': 'ukraine', 'severity': 'high'},
    '2024-11-19': {'event': 'Russia ICBM Strike Ukraine', 'color': '#d62728', 'category': 'ukraine', 'severity': 'critical'},
    '2025-03-01': {'event': 'Trump-Zelensky Meeting', 'color': '#17becf', 'category': 'ukraine', 'severity': 'high'},
    
    # Other Major Events
    '2023-08-24': {'event': 'BRICS Expansion Announced', 'color': '#e377c2', 'category': 'other', 'severity': 'medium'},
    '2024-01-13': {'event': 'Taiwan Election', 'color': '#e377c2', 'category': 'other', 'severity': 'high'},
    '2024-07-21': {'event': 'Biden Drops Out', 'color': '#1f77b4', 'category': 'other', 'severity': 'high'},
    '2024-12-04': {'event': 'South Korea Martial Law', 'color': '#d62728', 'category': 'other', 'severity': 'high'},
    '2024-12-08': {'event': 'Syria Assad Falls', 'color': '#d62728', 'category': 'other', 'severity': 'high'},
}

# Count by category
categories = {}
for date, info in GEOPOLITICAL_EVENTS.items():
    cat = info['category']
    categories[cat] = categories.get(cat, 0) + 1

print(f"✅ Loaded {len(GEOPOLITICAL_EVENTS)} geopolitical events:")
for cat, count in sorted(categories.items(), key=lambda x: -x[1]):
    print(f"   • {cat}: {count} events")

✅ Loaded 39 geopolitical events:
   • trump: 12 events
   • israel: 6 events
   • ukraine: 6 events
   • semiconductors: 5 events
   • other: 5 events
   • uk: 4 events
   • eu: 1 events


---
## 5️⃣ Data Download Functions

In [5]:
def download_tickers(tickers: dict, period: str = "2y", show_progress: bool = True) -> pd.DataFrame:
    """
    Download price data for a dictionary of tickers.
    
    Parameters:
    -----------
    tickers : dict
        Dictionary of {name: ticker_symbol}
    period : str
        Download period (e.g., '1y', '2y', '5y')
    show_progress : bool
        Whether to show download progress
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with prices for each ticker
    """
    all_data = {}
    
    for name, ticker in tickers.items():
        try:
            if show_progress:
                print(f"  📥 {name}...", end=" ")
            
            df = yf.download(ticker, period=period, progress=False, timeout=10)
            
            if not df.empty and len(df) > 20:
                col = 'Adj Close' if 'Adj Close' in df.columns else 'Close'
                all_data[name] = df[col].squeeze()
                if show_progress:
                    print("✓")
            else:
                if show_progress:
                    print("✗ (no data)")
                    
        except Exception as e:
            if show_progress:
                print(f"✗ ({str(e)[:20]}...)")
            
            # Try ETF alternative
            if name in ETF_ALTERNATIVES:
                try:
                    alt = ETF_ALTERNATIVES[name]
                    df = yf.download(alt, period=period, progress=False)
                    if not df.empty:
                        all_data[name] = df['Adj Close'].squeeze()
                        if show_progress:
                            print(f"      → Using {alt} ✓")
                except:
                    pass
    
    result = pd.DataFrame(all_data)
    result.index = pd.to_datetime(result.index)
    return result.ffill(limit=5)


def download_all_data(period: str = "2y") -> dict:
    """
    Download all required market data.
    
    Returns:
    --------
    dict
        Dictionary containing indices, factors, volatility, and sectors DataFrames
    """
    print("=" * 70)
    print("📊 DOWNLOADING MARKET DATA")
    print("=" * 70)
    
    data = {}
    
    print("\n🌍 International Indices:")
    data['indices'] = download_tickers(INDEX_TICKERS, period)
    
    print("\n🎯 Factor ETFs:")
    data['factors'] = download_tickers(FACTOR_TICKERS, period)
    
    print("\n📉 Volatility Indices:")
    data['volatility'] = download_tickers(VOLATILITY_TICKERS, period)
    
    print("\n📈 Sector ETFs:")
    data['sectors'] = download_tickers(SECTOR_TICKERS, period)
    
    print("\n" + "=" * 70)
    print("✅ DATA DOWNLOAD COMPLETE")
    print("=" * 70)
    
    return data

---
## 6️⃣ Download Data

⏱️ This may take 1-2 minutes depending on your connection.

In [6]:
# Download all data (2 years)
data = download_all_data(period="2y")

# Extract DataFrames
prices = data['indices']
factor_prices = data['factors']
vol_data = data['volatility']
sector_prices = data['sectors']

# Summary
print(f"\n📊 Data Summary:")
print(f"   • Indices: {len(prices.columns)} countries")
print(f"   • Date range: {prices.index.min().strftime('%Y-%m-%d')} to {prices.index.max().strftime('%Y-%m-%d')}")
print(f"   • Trading days: {len(prices)}")

📊 DOWNLOADING MARKET DATA

🌍 International Indices:
  📥 SPY... ✓
  📥 France... ✓
  📥 Germany... ✓
  📥 UK... ✓
  📥 Italy... ✓
  📥 Sweden... ✓
  📥 Spain... 

$^OBX: possibly delisted; no price data found  (period=2y)

1 Failed download:
['^OBX']: possibly delisted; no price data found  (period=2y)


✓
  📥 Norway... ✗ (no data)
  📥 Denmark... ✓
  📥 Switzerland... ✓
  📥 Finland... ✓
  📥 Korea... ✓
  📥 Japan... ✓
  📥 China... ✓
  📥 Hong Kong... ✓
  📥 Canada... ✓
  📥 Australia... ✓

🎯 Factor ETFs:
  📥 Market... ✓
  📥 Value... ✓
  📥 Growth... ✓
  📥 Momentum... ✓
  📥 Quality... ✓
  📥 Size... ✓
  📥 Low Vol... ✓

📉 Volatility Indices:
  📥 VIX... ✓
  📥 VIX9D... ✓
  📥 VIX3M... ✓
  📥 VIX6M... ✓
  📥 VVIX... ✓

📈 Sector ETFs:
  📥 US Tech... ✓
  📥 US Semiconductors... ✓
  📥 US Utilities... ✓
  📥 US Healthcare... ✓
  📥 Gold... ✓

✅ DATA DOWNLOAD COMPLETE

📊 Data Summary:
   • Indices: 16 countries
   • Date range: 2024-01-23 to 2026-01-23
   • Trading days: 522


---
## 7️⃣ Calculate Returns

In [7]:
def calculate_returns(prices: pd.DataFrame, frequency: str = 'daily') -> pd.DataFrame:
    """
    Calculate percentage returns at specified frequency.
    
    Parameters:
    -----------
    prices : pd.DataFrame
        Price data
    frequency : str
        'daily', 'weekly', or 'monthly'
        
    Returns:
    --------
    pd.DataFrame
        Returns in percentage terms
    """
    if frequency == 'weekly':
        prices = prices.resample('W-FRI').last()
    elif frequency == 'monthly':
        prices = prices.resample('M').last()
    
    return prices.pct_change().dropna() * 100


def calculate_realized_volatility(returns: pd.Series, window: int = 21) -> pd.Series:
    """
    Calculate annualized realized volatility.
    """
    return returns.rolling(window).std() * np.sqrt(252)


# Calculate returns
print("📈 Calculating returns...")

daily_returns = calculate_returns(prices, 'daily')
weekly_returns = calculate_returns(prices, 'weekly')
factor_returns = calculate_returns(factor_prices, 'weekly')

print(f"   • Daily returns: {len(daily_returns)} observations")
print(f"   • Weekly returns: {len(weekly_returns)} observations")
print(f"\n✅ Returns calculated!")

📈 Calculating returns...
   • Daily returns: 520 observations
   • Weekly returns: 104 observations

✅ Returns calculated!


---
## 8️⃣ Basic Statistics & Overview

In [8]:
def calculate_comprehensive_stats(returns: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate comprehensive statistics for all countries.
    """
    stats_list = []
    
    for country in returns.columns:
        if country == 'SPY':
            continue
        
        ret = returns[country].dropna()
        spy = returns['SPY'].loc[ret.index] if 'SPY' in returns.columns else None
        
        if len(ret) < 20:
            continue
        
        row = {
            'Country': country,
            'Region': COUNTRY_TO_REGION.get(country, 'Unknown'),
        }
        
        # Regression with SPY
        if spy is not None and len(spy) > 20:
            slope, intercept, r_val, p_val, std_err = stats.linregress(spy, ret)
            row.update({
                'Beta': slope,
                'Alpha (%)': intercept,
                'R²': r_val ** 2,
                'Correlation': r_val,
            })
        
        # Return stats
        row.update({
            'Avg Weekly (%)': ret.mean(),
            'Weekly Vol (%)': ret.std(),
            'Ann. Return (%)': ret.mean() * 52,
            'Ann. Vol (%)': ret.std() * np.sqrt(52),
            'Sharpe': (ret.mean() * 52) / (ret.std() * np.sqrt(52)) if ret.std() > 0 else 0,
        })
        
        # Risk metrics
        var_95 = np.percentile(ret, 5)
        cvar_95 = ret[ret <= var_95].mean()
        
        cumret = (1 + ret / 100).cumprod()
        max_dd = ((cumret / cumret.expanding().max()) - 1).min() * 100
        
        row.update({
            'VaR 95% (%)': var_95,
            'CVaR 95% (%)': cvar_95,
            'Max DD (%)': max_dd,
            'Skewness': ret.skew(),
            'Kurtosis': ret.kurtosis(),
        })
        
        stats_list.append(row)
    
    return pd.DataFrame(stats_list)


# Calculate statistics
stats_df = calculate_comprehensive_stats(weekly_returns)
stats_df = stats_df.sort_values('Sharpe', ascending=False)

# Display
print("📊 COUNTRY STATISTICS (Sorted by Sharpe Ratio)")
print("=" * 80)
display(stats_df.round(3))

📊 COUNTRY STATISTICS (Sorted by Sharpe Ratio)


,Country,Region,Beta,Alpha (%),R²,Correlation,Avg Weekly (%),Weekly Vol (%),Ann. Return (%),Ann. Vol (%),Sharpe,VaR 95% (%),CVaR 95% (%),Max DD (%),Skewness,Kurtosis
13,Canada,North America,0.535,0.243,0.563,0.750,0.445,1.471,23.130,10.610,2.180,-1.990,-3.087,-9.724,-0.988,3.678
5,Spain,Southern Europe,0.476,0.389,0.231,0.480,0.569,2.045,29.582,14.749,2.006,-2.683,-3.931,-7.971,-0.483,0.784
9,Korea,Asia Developed,0.553,0.499,0.192,0.438,0.709,2.604,36.846,18.780,1.962,-3.620,-4.399,-16.004,-0.125,-0.384
2,UK,Western Europe,0.358,0.148,0.264,0.514,0.284,1.439,14.759,10.377,1.422,-1.424,-2.742,-9.597,-0.888,5.422
1,Germany,Western Europe,0.579,0.172,0.340,0.583,0.391,2.049,20.321,14.777,1.375,-3.181,-4.201,-11.451,-0.639,1.998
3,Italy,Southern Europe,0.559,0.191,0.244,0.494,0.402,2.334,20.921,16.828,1.243,-3.130,-5.208,-12.829,-1.254,4.003
12,Hong Kong,Asia Developed,0.366,0.410,0.055,0.235,0.549,3.216,28.535,23.191,1.230,-4.664,-5.998,-16.153,0.535,2.473
11,China,Asia Emerging,0.188,0.295,0.027,0.164,0.366,2.373,19.023,17.109,1.112,-3.099,-4.304,-14.280,1.438,7.924
10,Japan,Asia Developed,0.813,0.127,0.344,0.587,0.435,2.863,22.633,20.647,1.096,-4.165,-5.979,-18.463,-0.247,0.873
8,Finland,Northern Europe,0.445,0.084,0.239,0.489,0.253,1.882,13.148,13.569,0.969,-2.424,-3.845,-13.522,-0.490,2.072


---
# 📊 SECTION A: Options-Implied Volatility Analysis

Analyzing VIX term structure, IV-RV spread, and volatility risk premium.

In [9]:
def analyze_implied_volatility(vol_data: pd.DataFrame, spy_returns: pd.Series) -> dict:
    """
    Comprehensive implied volatility analysis.
    
    Returns:
    --------
    dict containing:
        - vix_stats: VIX summary statistics
        - vix_percentile: Rolling percentile rank
        - iv_rv_spread: IV minus RV spread
        - vrp_stats: Volatility risk premium stats
        - term_structure: VIX term structure data
        - contango: Term structure slope
    """
    results = {}
    
    if 'VIX' not in vol_data.columns:
        print("⚠️ VIX data not available")
        return results
    
    vix = vol_data['VIX'].dropna()
    
    # VIX Statistics
    results['vix_stats'] = {
        'current': vix.iloc[-1] if len(vix) > 0 else np.nan,
        'mean': vix.mean(),
        'median': vix.median(),
        'std': vix.std(),
        'min': vix.min(),
        'max': vix.max(),
        'pct_25': vix.quantile(0.25),
        'pct_75': vix.quantile(0.75),
        'pct_90': vix.quantile(0.90),
        'pct_95': vix.quantile(0.95),
    }
    
    # VIX Percentile (1-year rolling)
    results['vix_percentile'] = vix.rolling(252).apply(
        lambda x: stats.percentileofscore(x, x.iloc[-1]) if len(x) > 50 else np.nan
    )
    
    # IV-RV Spread (Volatility Risk Premium)
    rv = calculate_realized_volatility(spy_returns, window=21)
    common_idx = vix.index.intersection(rv.index)
    
    if len(common_idx) > 50:
        results['iv_rv_spread'] = pd.DataFrame({
            'VIX': vix.loc[common_idx],
            'Realized Vol': rv.loc[common_idx],
            'Spread': vix.loc[common_idx] - rv.loc[common_idx]
        })
        results['vrp_stats'] = {
            'mean': results['iv_rv_spread']['Spread'].mean(),
            'current': results['iv_rv_spread']['Spread'].iloc[-1],
            'pct_positive': (results['iv_rv_spread']['Spread'] > 0).mean() * 100
        }
    
    # Term Structure
    term_cols = ['VIX9D', 'VIX', 'VIX3M', 'VIX6M']
    available = [c for c in term_cols if c in vol_data.columns]
    if len(available) > 1:
        results['term_structure'] = vol_data[available].dropna()
        
        # Contango/Backwardation (3M - 1M slope)
        if 'VIX3M' in vol_data.columns:
            spread = vol_data['VIX3M'] - vol_data['VIX']
            results['contango'] = spread
            results['contango_pct'] = (spread > 0).mean() * 100
    
    # VVIX
    if 'VVIX' in vol_data.columns:
        results['vvix'] = vol_data['VVIX'].dropna()
    
    return results


# Run analysis
print("📊 Analyzing Implied Volatility...")
vol_analysis = analyze_implied_volatility(vol_data, daily_returns.get('SPY', pd.Series()))

# Display summary
if 'vix_stats' in vol_analysis:
    vs = vol_analysis['vix_stats']
    print(f"\n📉 VIX Summary:")
    print(f"   • Current: {vs['current']:.1f}")
    print(f"   • Mean: {vs['mean']:.1f}")
    print(f"   • Min/Max: {vs['min']:.1f} / {vs['max']:.1f}")
    print(f"   • 90th Percentile: {vs['pct_90']:.1f}")

if 'vrp_stats' in vol_analysis:
    vrp = vol_analysis['vrp_stats']
    print(f"\n💰 Volatility Risk Premium:")
    print(f"   • Current: {vrp['current']:.1f}%")
    print(f"   • Mean: {vrp['mean']:.1f}%")
    print(f"   • Positive {vrp['pct_positive']:.0f}% of the time")

📊 Analyzing Implied Volatility...

📉 VIX Summary:
   • Current: 16.1
   • Mean: 17.4
   • Min/Max: 11.9 / 52.3
   • 90th Percentile: 22.3

💰 Volatility Risk Premium:
   • Current: 5.9%
   • Mean: 3.6%
   • Positive 85% of the time


In [10]:
# Create Implied Volatility Dashboard
def create_implied_vol_dashboard(vol_analysis: dict, vol_data: pd.DataFrame,
                                  spy_returns: pd.Series, events: dict) -> go.Figure:
    """
    Create comprehensive implied volatility dashboard.
    """
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'VIX Term Structure Over Time',
            'VIX Distribution',
            'Implied vs Realized Volatility',
            'Volatility Risk Premium (IV - RV)',
            'VIX Percentile Rank (1-Year Rolling)',
            'VIX Regime Classification'
        ),
        vertical_spacing=0.1,
        horizontal_spacing=0.08,
        specs=[[{'type': 'scatter'}, {'type': 'histogram'}],
               [{'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'scatter'}, {'type': 'domain'}]]
    )
    
    vix = vol_data.get('VIX', pd.Series())
    if vix.empty:
        return fig
    vix = vix.dropna()
    
    # 1. Term Structure
    colors = {'VIX9D': '#ff7f0e', 'VIX': '#d62728', 'VIX3M': '#1f77b4', 'VIX6M': '#2ca02c'}
    for col in ['VIX9D', 'VIX', 'VIX3M', 'VIX6M']:
        if col in vol_data.columns:
            series = vol_data[col].dropna()
            fig.add_trace(go.Scatter(
                x=series.index, y=series,
                mode='lines', name=col,
                line=dict(color=colors.get(col), width=1.5)
            ), row=1, col=1)
    
    # Add regime bands
    for y0, y1, color, label in [(0, 15, 'green', 'Low'), (15, 20, 'yellow', 'Normal'),
                                  (20, 30, 'orange', 'Elevated'), (30, 80, 'red', 'High')]:
        fig.add_hrect(y0=y0, y1=y1, fillcolor=color, opacity=0.1, row=1, col=1)
    
    # 2. VIX Distribution
    fig.add_trace(go.Histogram(
        x=vix, nbinsx=50, name='VIX',
        marker_color='#1f77b4', opacity=0.7, showlegend=False
    ), row=1, col=2)
    fig.add_vline(x=vix.mean(), line_dash="dash", line_color="red", row=1, col=2)
    fig.add_vline(x=vix.iloc[-1], line_dash="solid", line_color="green", row=1, col=2)
    
    # 3. IV vs RV
    if 'iv_rv_spread' in vol_analysis:
        spread_df = vol_analysis['iv_rv_spread']
        fig.add_trace(go.Scatter(
            x=spread_df.index, y=spread_df['VIX'],
            mode='lines', name='Implied (VIX)',
            line=dict(color='#d62728', width=2)
        ), row=2, col=1)
        fig.add_trace(go.Scatter(
            x=spread_df.index, y=spread_df['Realized Vol'],
            mode='lines', name='Realized (21d)',
            line=dict(color='#1f77b4', width=2)
        ), row=2, col=1)
        
        # 4. VRP
        fig.add_trace(go.Scatter(
            x=spread_df.index, y=spread_df['Spread'],
            mode='lines', name='IV-RV Spread',
            line=dict(color='#2ca02c', width=2),
            fill='tozeroy', fillcolor='rgba(44, 160, 44, 0.2)',
            showlegend=False
        ), row=2, col=2)
        fig.add_hline(y=0, line_dash="solid", line_color="gray", row=2, col=2)
        if 'vrp_stats' in vol_analysis:
            fig.add_hline(y=vol_analysis['vrp_stats']['mean'], line_dash="dash",
                         line_color="red", row=2, col=2)
    
    # 5. VIX Percentile
    if 'vix_percentile' in vol_analysis:
        pctl = vol_analysis['vix_percentile'].dropna()
        fig.add_trace(go.Scatter(
            x=pctl.index, y=pctl,
            mode='lines', name='Percentile',
            line=dict(color='#9467bd', width=2),
            fill='tozeroy', fillcolor='rgba(148, 103, 189, 0.2)',
            showlegend=False
        ), row=3, col=1)
        fig.add_hline(y=80, line_dash="dash", line_color="red", row=3, col=1)
        fig.add_hline(y=20, line_dash="dash", line_color="green", row=3, col=1)
    
    # 6. Regime Pie - use 'domain' type for pie charts in subplots
    regime_counts = {
        'Low (<15)': (vix < 15).sum(),
        'Normal (15-20)': ((vix >= 15) & (vix < 20)).sum(),
        'Elevated (20-30)': ((vix >= 20) & (vix < 30)).sum(),
        'High (>30)': (vix >= 30).sum()
    }
    fig.add_trace(go.Pie(
        labels=list(regime_counts.keys()),
        values=list(regime_counts.values()),
        marker_colors=['#2ca02c', '#ffff00', '#ff7f0e', '#d62728'],
        textinfo='percent+label',
        hole=0.4,
        showlegend=False
    ), row=3, col=2)
    
    # Add events - use add_shape instead of add_vline for compatibility with mixed subplot types
    for date_str, event_info in events.items():
        event_date = pd.to_datetime(date_str)
        if vix.index.min() <= event_date <= vix.index.max():
            if event_info.get('severity') in ['high', 'critical']:
                # Add vertical line to row 1 col 1 using add_shape
                fig.add_shape(
                    type="line",
                    x0=event_date, x1=event_date,
                    y0=0, y1=1,
                    yref='y domain',
                    line=dict(color=event_info['color'], width=1, dash='dash'),
                    opacity=0.4,
                    row=1, col=1
                )
                # Add vertical line to row 2 col 1 using add_shape
                fig.add_shape(
                    type="line",
                    x0=event_date, x1=event_date,
                    y0=0, y1=1,
                    yref='y3 domain',
                    line=dict(color=event_info['color'], width=1, dash='dash'),
                    opacity=0.4,
                    row=2, col=1
                )
    
    fig.update_layout(
        title=dict(
            text='<b>Options-Implied Volatility Analysis</b><br>'
                 '<sup>VIX Term Structure | IV-RV Spread | Volatility Risk Premium</sup>',
            x=0.5, font=dict(size=18)
        ),
        height=1000, width=1200,
        showlegend=True,
        legend=dict(orientation='h', y=1.02),
        template='plotly_white'
    )
    
    return fig


# Create and display
fig_iv = create_implied_vol_dashboard(
    vol_analysis, vol_data, 
    daily_returns.get('SPY', pd.Series()), 
    GEOPOLITICAL_EVENTS
)
fig_iv.show()

---
# 🔗 SECTION B: Correlation Regime Analysis

Analyzing how correlations change between low and high volatility regimes.

In [11]:
def calculate_correlation_by_regime(returns: pd.DataFrame, vix: pd.Series,
                                     thresholds: tuple = (20, 30)) -> dict:
    """
    Calculate correlation matrices for different VIX regimes.
    
    Parameters:
    -----------
    returns : pd.DataFrame
        Weekly returns
    vix : pd.Series
        VIX time series
    thresholds : tuple
        (low_threshold, high_threshold) for regime classification
        
    Returns:
    --------
    dict with correlation matrices for each regime
    """
    common_idx = returns.index.intersection(vix.index)
    returns_aligned = returns.loc[common_idx]
    vix_aligned = vix.loc[common_idx]
    
    low_vol = vix_aligned < thresholds[0]
    mid_vol = (vix_aligned >= thresholds[0]) & (vix_aligned < thresholds[1])
    high_vol = vix_aligned >= thresholds[1]
    
    results = {
        'Low VIX (<20)': {
            'corr': returns_aligned[low_vol].corr(),
            'count': low_vol.sum(),
            'pct': low_vol.mean() * 100
        },
        'Normal VIX (20-30)': {
            'corr': returns_aligned[mid_vol].corr(),
            'count': mid_vol.sum(),
            'pct': mid_vol.mean() * 100
        },
        'High VIX (>30)': {
            'corr': returns_aligned[high_vol].corr(),
            'count': high_vol.sum(),
            'pct': high_vol.mean() * 100
        },
        'Full Period': {
            'corr': returns_aligned.corr(),
            'count': len(returns_aligned),
            'pct': 100.0
        }
    }
    
    # Correlation change
    if low_vol.sum() > 10 and high_vol.sum() > 10:
        results['corr_change'] = results['High VIX (>30)']['corr'] - results['Low VIX (<20)']['corr']
    
    return results


# Calculate correlation regimes
vix = vol_data.get('VIX', pd.Series())
if not vix.empty:
    print("🔗 Calculating correlation regimes...")
    corr_regimes = calculate_correlation_by_regime(weekly_returns, vix)
    
    print(f"\n📊 Regime Distribution:")
    for regime, data in corr_regimes.items():
        if regime != 'corr_change' and 'pct' in data:
            print(f"   • {regime}: {data['pct']:.1f}% ({data['count']} days)")
else:
    corr_regimes = {}
    print("⚠️ VIX data not available")

🔗 Calculating correlation regimes...

📊 Regime Distribution:
   • Low VIX (<20): 79.2% (80 days)
   • Normal VIX (20-30): 18.8% (19 days)
   • High VIX (>30): 2.0% (2 days)
   • Full Period: 100.0% (101 days)


In [12]:
# Create Correlation Regime Heatmaps
def create_correlation_regime_heatmaps(corr_regimes: dict) -> go.Figure:
    """
    Create heatmaps for different correlation regimes.
    """
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            f"Low VIX (<20) - {corr_regimes.get('Low VIX (<20)', {}).get('pct', 0):.1f}% of days",
            f"High VIX (>30) - {corr_regimes.get('High VIX (>30)', {}).get('pct', 0):.1f}% of days",
            'Correlation CHANGE (High - Low VIX)',
            'Full Period Correlation'
        ],
        horizontal_spacing=0.12,
        vertical_spacing=0.15
    )
    
    top_countries = ['SPY', 'Germany', 'UK', 'France', 'Japan', 'China', 'Canada', 'Australia']
    
    def add_heatmap(corr_matrix, row, col, zmid=0.5, colorscale='RdBu_r'):
        if corr_matrix is None or corr_matrix.empty:
            return
        available = [c for c in top_countries if c in corr_matrix.columns]
        if not available:
            return
        sub = corr_matrix.loc[available, available]
        
        fig.add_trace(go.Heatmap(
            z=sub.values, x=sub.columns, y=sub.index,
            colorscale=colorscale, 
            zmin=-1 if zmid == 0 else 0, 
            zmax=1,
            zmid=zmid,
            text=np.round(sub.values, 2),
            texttemplate='%{text}',
            textfont={"size": 9},
            showscale=(row == 1 and col == 2)
        ), row=row, col=col)
    
    # Low VIX
    if 'Low VIX (<20)' in corr_regimes:
        add_heatmap(corr_regimes['Low VIX (<20)']['corr'], 1, 1)
    
    # High VIX
    if 'High VIX (>30)' in corr_regimes:
        add_heatmap(corr_regimes['High VIX (>30)']['corr'], 1, 2)
    
    # Change (Red = correlations increased during stress)
    if 'corr_change' in corr_regimes:
        add_heatmap(corr_regimes['corr_change'], 2, 1, zmid=0, colorscale='RdYlGn_r')
    
    # Full period
    if 'Full Period' in corr_regimes:
        add_heatmap(corr_regimes['Full Period']['corr'], 2, 2)
    
    fig.update_layout(
        title=dict(
            text='<b>🔗 Correlation Matrix by VIX Regime</b><br>'
                 '<sup>⚠️ Notice: Correlations INCREASE during high volatility (diversification breakdown)</sup>',
            x=0.5, font=dict(size=18)
        ),
        height=900, width=1100,
        template='plotly_white'
    )
    
    return fig


# Create and display
if corr_regimes:
    fig_corr = create_correlation_regime_heatmaps(corr_regimes)
    fig_corr.show()

In [13]:
# Correlation Change Bar Chart
def create_correlation_change_chart(corr_regimes: dict) -> go.Figure:
    """
    Bar chart comparing correlations with SPY in different regimes.
    """
    if 'Low VIX (<20)' not in corr_regimes or 'High VIX (>30)' not in corr_regimes:
        return None
    
    low_corr = corr_regimes['Low VIX (<20)']['corr']
    high_corr = corr_regimes['High VIX (>30)']['corr']
    
    countries = ['Germany', 'UK', 'France', 'Japan', 'China', 'Canada', 'Australia', 'Korea', 'Italy']
    data = []
    
    for c in countries:
        if c in low_corr.columns and c in high_corr.columns and 'SPY' in low_corr.columns:
            data.append({
                'Country': c,
                'Low VIX': low_corr.loc[c, 'SPY'] if c != 'SPY' else 1.0,
                'High VIX': high_corr.loc[c, 'SPY'] if c != 'SPY' else 1.0,
            })
    
    if not data:
        return None
    
    df = pd.DataFrame(data)
    df['Change'] = df['High VIX'] - df['Low VIX']
    df = df.sort_values('Change', ascending=False)
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=df['Country'], y=df['Low VIX'],
        name='Low VIX (<20)', marker_color='#2ca02c'
    ))
    fig.add_trace(go.Bar(
        x=df['Country'], y=df['High VIX'],
        name='High VIX (>30)', marker_color='#d62728'
    ))
    
    fig.update_layout(
        title=dict(
            text='<b>Correlation with SPY: Low vs High VIX Regimes</b><br>'
                 '<sup>All correlations increase during market stress (diversification fails when needed most)</sup>',
            x=0.5, font=dict(size=16)
        ),
        barmode='group',
        yaxis_title='Correlation with SPY',
        height=500, width=1000,
        template='plotly_white'
    )
    
    return fig


if corr_regimes:
    fig_corr_change = create_correlation_change_chart(corr_regimes)
    if fig_corr_change:
        fig_corr_change.show()

---
# 🎯 SECTION C: Factor Decomposition

Decomposing country returns into factor exposures: Market, Value, Momentum, Quality, Size, Low Vol.

In [14]:
def calculate_factor_exposures(country_returns: pd.DataFrame,
                                factor_returns: pd.DataFrame,
                                window: int = 52) -> dict:
    """
    Calculate factor exposures using OLS regression.
    
    Returns:
    --------
    dict: {country: {alpha, r_squared, betas, t_stats, rolling_betas}}
    """
    if not STATSMODELS_AVAILABLE:
        print("⚠️ statsmodels not available - skipping factor analysis")
        return {}
    
    factors = ['Market', 'Value', 'Momentum', 'Quality', 'Size', 'Low Vol']
    available_factors = [f for f in factors if f in factor_returns.columns]
    
    if len(available_factors) < 2:
        return {}
    
    results = {}
    
    for country in country_returns.columns:
        if country == 'SPY':
            continue
        
        common_idx = country_returns.index.intersection(factor_returns.index)
        y = country_returns.loc[common_idx, country]
        X = factor_returns.loc[common_idx, available_factors]
        
        mask = y.notna() & X.notna().all(axis=1)
        y_clean = y[mask]
        X_clean = X[mask]
        
        if len(y_clean) < 50:
            continue
        
        try:
            X_const = add_constant(X_clean)
            model = OLS(y_clean, X_const).fit()
            
            results[country] = {
                'alpha': model.params.get('const', 0),
                'r_squared': model.rsquared,
                'betas': {f: model.params.get(f, 0) for f in available_factors},
                't_stats': {f: model.tvalues.get(f, 0) for f in available_factors},
            }
            
            # Rolling betas for Market and Momentum
            results[country]['rolling_betas'] = {}
            for factor in ['Market', 'Momentum']:
                if factor in available_factors:
                    rolling_beta = []
                    for i in range(window, len(y_clean)):
                        y_w = y_clean.iloc[i-window:i]
                        X_w = add_constant(X_clean.iloc[i-window:i])
                        try:
                            m = OLS(y_w, X_w).fit()
                            rolling_beta.append({
                                'date': y_clean.index[i],
                                'beta': m.params.get(factor, np.nan)
                            })
                        except:
                            pass
                    
                    if rolling_beta:
                        results[country]['rolling_betas'][factor] = \
                            pd.DataFrame(rolling_beta).set_index('date')['beta']
                        
        except Exception as e:
            pass
    
    return results


# Calculate factor exposures
print("🎯 Calculating factor exposures...")
factor_exposures = calculate_factor_exposures(weekly_returns, factor_returns)

print(f"\n✅ Factor analysis completed for {len(factor_exposures)} countries")

# Display summary
if factor_exposures:
    print("\n📊 Factor Exposure Summary:")
    for country in list(factor_exposures.keys())[:5]:
        exp = factor_exposures[country]
        print(f"\n   {country}:")
        print(f"      R² = {exp['r_squared']:.1%}")
        print(f"      Market β = {exp['betas'].get('Market', 0):.2f}")
        print(f"      Momentum β = {exp['betas'].get('Momentum', 0):.2f}")

🎯 Calculating factor exposures...

✅ Factor analysis completed for 15 countries

📊 Factor Exposure Summary:

   France:
      R² = 40.5%
      Market β = -0.25
      Momentum β = 0.02

   Germany:
      R² = 39.8%
      Market β = -0.47
      Momentum β = 0.19

   UK:
      R² = 35.4%
      Market β = -0.37
      Momentum β = 0.16

   Italy:
      R² = 36.7%
      Market β = -0.09
      Momentum β = 0.19

   Sweden:
      R² = 53.3%
      Market β = -0.26
      Momentum β = -0.04


In [15]:
# Factor Exposure Heatmap
def create_factor_exposure_heatmap(factor_exposures: dict) -> go.Figure:
    """
    Create heatmap of factor exposures across countries.
    """
    data = []
    for country, exp in factor_exposures.items():
        row = {'Country': country, 'Alpha': exp['alpha'], 'R²': exp['r_squared']}
        row.update(exp['betas'])
        data.append(row)
    
    if not data:
        return None
    
    df = pd.DataFrame(data).sort_values('R²', ascending=False)
    
    factor_cols = [c for c in df.columns if c not in ['Country', 'Alpha', 'R²']]
    
    fig = go.Figure(data=go.Heatmap(
        z=df[factor_cols].values,
        x=factor_cols,
        y=df['Country'],
        colorscale='RdBu_r',
        zmid=0 if 'Market' not in factor_cols else 1,
        text=np.round(df[factor_cols].values, 2),
        texttemplate='%{text}',
        textfont={"size": 10},
        colorbar=dict(title='Beta')
    ))
    
    fig.update_layout(
        title=dict(
            text='<b>🎯 Factor Exposures by Country</b><br>'
                 '<sup>Market, Value, Momentum, Quality, Size, Low Vol | Red=Positive, Blue=Negative</sup>',
            x=0.5, font=dict(size=16)
        ),
        height=550, width=950,
        template='plotly_white'
    )
    
    return fig


if factor_exposures:
    fig_factors = create_factor_exposure_heatmap(factor_exposures)
    if fig_factors:
        fig_factors.show()

---
# 🎲 SECTION D: Monte Carlo Simulations

Forward-looking scenarios using Geometric Brownian Motion, VaR, CVaR, and stress testing.

In [16]:
def run_monte_carlo(returns: pd.Series, n_sims: int = 10000, n_days: int = 252,
                    initial_price: float = 100) -> dict:
    """
    Run Monte Carlo simulation using Geometric Brownian Motion.
    
    Parameters:
    -----------
    returns : pd.Series
        Daily returns in percentage
    n_sims : int
        Number of simulations
    n_days : int
        Number of trading days to simulate
    initial_price : float
        Starting price
        
    Returns:
    --------
    dict containing price_paths, final_returns, stats, and params
    """
    # Estimate parameters from historical data
    daily_mean = returns.mean() / 100
    daily_std = returns.std() / 100
    
    # Annualize
    annual_return = daily_mean * 252
    annual_vol = daily_std * np.sqrt(252)
    
    dt = 1 / 252
    
    # Generate random paths using GBM
    np.random.seed(42)  # For reproducibility
    random_matrix = np.random.standard_normal((n_days, n_sims))
    
    drift = (annual_return - 0.5 * annual_vol**2) * dt
    diffusion = annual_vol * np.sqrt(dt) * random_matrix
    
    daily_returns_sim = drift + diffusion
    
    # Convert to price paths
    price_paths = initial_price * np.exp(np.cumsum(daily_returns_sim, axis=0))
    price_paths = np.vstack([np.full(n_sims, initial_price), price_paths])
    
    # Calculate final statistics
    final_prices = price_paths[-1, :]
    final_returns = (final_prices / initial_price - 1) * 100
    
    results = {
        'price_paths': price_paths,
        'final_returns': final_returns,
        'stats': {
            'mean': final_returns.mean(),
            'median': np.median(final_returns),
            'std': final_returns.std(),
            'var_95': np.percentile(final_returns, 5),
            'var_99': np.percentile(final_returns, 1),
            'cvar_95': final_returns[final_returns <= np.percentile(final_returns, 5)].mean(),
            'cvar_99': final_returns[final_returns <= np.percentile(final_returns, 1)].mean(),
            'prob_positive': (final_returns > 0).mean() * 100,
            'prob_10_gain': (final_returns > 10).mean() * 100,
            'prob_10_loss': (final_returns < -10).mean() * 100,
            'prob_20_loss': (final_returns < -20).mean() * 100,
            'percentiles': {
                '5th': np.percentile(final_returns, 5),
                '25th': np.percentile(final_returns, 25),
                '50th': np.percentile(final_returns, 50),
                '75th': np.percentile(final_returns, 75),
                '95th': np.percentile(final_returns, 95),
            }
        },
        'params': {
            'annual_return': annual_return,
            'annual_vol': annual_vol
        }
    }
    
    return results


def run_stress_scenarios(returns: pd.Series, initial_price: float = 100) -> dict:
    """
    Run stress test scenarios.
    """
    daily_mean = returns.mean() / 100
    daily_std = returns.std() / 100
    
    annual_return = daily_mean * 252
    annual_vol = daily_std * np.sqrt(252)
    
    scenarios = {
        'Base Case': {'mu': annual_return, 'sigma': annual_vol},
        'Bull Market': {'mu': annual_return + 0.10, 'sigma': annual_vol * 0.8},
        'Bear Market': {'mu': annual_return - 0.15, 'sigma': annual_vol * 1.3},
        'High Vol': {'mu': annual_return * 0.5, 'sigma': annual_vol * 2.0},
        'Low Vol': {'mu': annual_return * 1.2, 'sigma': annual_vol * 0.6},
        'Market Crash': {'mu': -0.35, 'sigma': annual_vol * 2.5},
        'Recovery': {'mu': 0.30, 'sigma': annual_vol * 1.5},
    }
    
    results = {}
    n_sims = 1000
    n_days = 252
    dt = 1 / 252
    
    for name, params in scenarios.items():
        np.random.seed(42)
        mu, sigma = params['mu'], params['sigma']
        
        random_matrix = np.random.standard_normal((n_days, n_sims))
        drift = (mu - 0.5 * sigma**2) * dt
        diffusion = sigma * np.sqrt(dt) * random_matrix
        
        price_paths = initial_price * np.exp(np.cumsum(drift + diffusion, axis=0))
        final_returns = (price_paths[-1, :] / initial_price - 1) * 100
        
        results[name] = {
            'mean_return': final_returns.mean(),
            'var_95': np.percentile(final_returns, 5),
            'prob_positive': (final_returns > 0).mean() * 100,
            'max_loss': final_returns.min(),
        }
    
    return results


# Run Monte Carlo for key markets
print("🎲 Running Monte Carlo simulations (10,000 paths each)...")

mc_results = {}
stress_results = {}

for country in ['SPY', 'Germany', 'UK', 'Japan', 'China', 'France']:
    if country in daily_returns.columns:
        ret = daily_returns[country].dropna()
        if len(ret) > 100:
            print(f"   • {country}...", end=" ")
            mc_results[country] = run_monte_carlo(ret)
            stress_results[country] = run_stress_scenarios(ret)
            print("✓")

print(f"\n✅ Monte Carlo completed for {len(mc_results)} markets")

🎲 Running Monte Carlo simulations (10,000 paths each)...
   • SPY... ✓
   • Germany... ✓
   • UK... ✓
   • Japan... ✓
   • China... ✓
   • France... ✓

✅ Monte Carlo completed for 6 markets


In [17]:
# Display Monte Carlo Summary
print("\n📊 MONTE CARLO SIMULATION RESULTS (1-Year Forward)")
print("=" * 80)

mc_summary = []
for country, mc in mc_results.items():
    s = mc['stats']
    p = mc['params']
    mc_summary.append({
        'Country': country,
        'Ann. Return': f"{p['annual_return']:.1%}",
        'Ann. Vol': f"{p['annual_vol']:.1%}",
        'Expected': f"{s['mean']:.1f}%",
        'VaR 95%': f"{s['var_95']:.1f}%",
        'CVaR 95%': f"{s['cvar_95']:.1f}%",
        'P(+)': f"{s['prob_positive']:.0f}%",
        'P(>10%)': f"{s['prob_10_gain']:.0f}%",
        'P(<-20%)': f"{s['prob_20_loss']:.0f}%",
    })

display(pd.DataFrame(mc_summary))


📊 MONTE CARLO SIMULATION RESULTS (1-Year Forward)


,Country,Ann. Return,Ann. Vol,Expected,VaR 95%,CVaR 95%,P(+),P(>10%),P(<-20%)
0,SPY,19.5%,16.1%,21.3%,-8.0%,-13.8%,87%,70%,1%
1,Germany,19.9%,14.9%,21.9%,-5.5%,-11.1%,89%,73%,0%
2,UK,15.0%,10.6%,16.1%,-2.9%,-7.1%,91%,68%,0%
3,Japan,22.2%,24.4%,24.6%,-18.8%,-26.5%,78%,65%,4%
4,China,19.9%,16.3%,21.8%,-7.9%,-13.9%,87%,71%,1%
5,France,5.3%,14.1%,5.3%,-17.3%,-22.0%,62%,35%,3%


In [18]:
# Monte Carlo Dashboard for SPY
def create_monte_carlo_dashboard(mc_results: dict, country: str) -> go.Figure:
    """
    Create comprehensive Monte Carlo visualization.
    """
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            f'{country}: Simulated Price Paths (200 of 10,000)',
            'Distribution of 1-Year Returns',
            'Return Percentiles',
            'Risk Metrics Summary'
        ),
        specs=[[{'type': 'scatter'}, {'type': 'histogram'}],
               [{'type': 'bar'}, {'type': 'table'}]]
    )
    
    paths = mc_results['price_paths']
    stats = mc_results['stats']
    params = mc_results['params']
    
    # 1. Sample paths
    n_show = min(200, paths.shape[1])
    for i in range(n_show):
        fig.add_trace(go.Scatter(
            y=paths[:, i],
            mode='lines',
            line=dict(width=0.3, color='rgba(31, 119, 180, 0.15)'),
            showlegend=False
        ), row=1, col=1)
    
    # Percentile bands
    p5 = np.percentile(paths, 5, axis=1)
    p50 = np.percentile(paths, 50, axis=1)
    p95 = np.percentile(paths, 95, axis=1)
    
    fig.add_trace(go.Scatter(y=p50, mode='lines', name='Median',
                             line=dict(color='red', width=2.5)), row=1, col=1)
    fig.add_trace(go.Scatter(y=p5, mode='lines', name='5th %ile',
                             line=dict(color='green', width=2, dash='dash')), row=1, col=1)
    fig.add_trace(go.Scatter(y=p95, mode='lines', name='95th %ile',
                             line=dict(color='orange', width=2, dash='dash')), row=1, col=1)
    
    # 2. Histogram
    fig.add_trace(go.Histogram(
        x=mc_results['final_returns'],
        nbinsx=75,
        marker_color='#1f77b4',
        opacity=0.7,
        showlegend=False
    ), row=1, col=2)
    
    fig.add_vline(x=stats['var_95'], line_dash="dash", line_color="red", row=1, col=2)
    fig.add_vline(x=stats['var_99'], line_dash="dash", line_color="darkred", row=1, col=2)
    fig.add_vline(x=0, line_dash="solid", line_color="black", row=1, col=2)
    
    # 3. Percentiles bar
    pctl = stats['percentiles']
    fig.add_trace(go.Bar(
        x=list(pctl.keys()),
        y=list(pctl.values()),
        marker_color=['#d62728', '#ff7f0e', '#2ca02c', '#ff7f0e', '#d62728'],
        text=[f'{v:.1f}%' for v in pctl.values()],
        textposition='outside',
        showlegend=False
    ), row=2, col=1)
    fig.add_hline(y=0, line_dash="solid", line_color="black", row=2, col=1)
    
    # 4. Table
    metrics = [
        ['Expected Return', f"{stats['mean']:.1f}%"],
        ['Median Return', f"{stats['median']:.1f}%"],
        ['Volatility', f"{stats['std']:.1f}%"],
        ['VaR (95%)', f"{stats['var_95']:.1f}%"],
        ['CVaR (95%)', f"{stats['cvar_95']:.1f}%"],
        ['VaR (99%)', f"{stats['var_99']:.1f}%"],
        ['P(Return > 0)', f"{stats['prob_positive']:.0f}%"],
        ['P(Return > 10%)', f"{stats['prob_10_gain']:.0f}%"],
        ['P(Return < -10%)', f"{stats['prob_10_loss']:.0f}%"],
        ['P(Return < -20%)', f"{stats['prob_20_loss']:.0f}%"],
    ]
    
    fig.add_trace(go.Table(
        header=dict(
            values=['<b>Metric</b>', '<b>Value</b>'],
            fill_color='#1f77b4',
            font=dict(color='white', size=11),
            align='left'
        ),
        cells=dict(
            values=[[m[0] for m in metrics], [m[1] for m in metrics]],
            fill_color='white',
            align='left'
        )
    ), row=2, col=2)
    
    fig.update_layout(
        title=dict(
            text=f'<b>🎲 {country} Monte Carlo Simulation (1-Year Forward)</b><br>'
                 f'<sup>μ = {params["annual_return"]:.1%}/yr | σ = {params["annual_vol"]:.1%}/yr | 10,000 simulations</sup>',
            x=0.5, font=dict(size=16)
        ),
        height=800, width=1200,
        template='plotly_white'
    )
    
    fig.update_xaxes(title_text='Trading Days', row=1, col=1)
    fig.update_yaxes(title_text='Price', row=1, col=1)
    fig.update_xaxes(title_text='1-Year Return (%)', row=1, col=2)
    fig.update_yaxes(title_text='Return (%)', row=2, col=1)
    
    return fig


# Show SPY Monte Carlo
if 'SPY' in mc_results:
    fig_mc_spy = create_monte_carlo_dashboard(mc_results['SPY'], 'SPY')
    fig_mc_spy.show()

In [19]:
# Show Germany Monte Carlo
if 'Germany' in mc_results:
    fig_mc_ger = create_monte_carlo_dashboard(mc_results['Germany'], 'Germany')
    fig_mc_ger.show()

In [20]:
# Stress Test Visualization
def create_stress_test_chart(stress_results: dict, country: str) -> go.Figure:
    """
    Visualize stress test scenarios.
    """
    scenarios = list(stress_results.keys())
    mean_returns = [stress_results[s]['mean_return'] for s in scenarios]
    var_95 = [stress_results[s]['var_95'] for s in scenarios]
    prob_pos = [stress_results[s]['prob_positive'] for s in scenarios]
    
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Expected Return', 'Value at Risk (95%)', 'P(Positive Return)')
    )
    
    # Mean returns
    colors_mean = ['#2ca02c' if r > 0 else '#d62728' for r in mean_returns]
    fig.add_trace(go.Bar(
        x=scenarios, y=mean_returns,
        marker_color=colors_mean,
        text=[f'{r:.0f}%' for r in mean_returns],
        textposition='outside',
        showlegend=False
    ), row=1, col=1)
    
    # VaR
    fig.add_trace(go.Bar(
        x=scenarios, y=var_95,
        marker_color='#d62728',
        text=[f'{r:.0f}%' for r in var_95],
        textposition='outside',
        showlegend=False
    ), row=1, col=2)
    
    # Prob positive
    colors_prob = ['#2ca02c' if p > 50 else '#d62728' for p in prob_pos]
    fig.add_trace(go.Bar(
        x=scenarios, y=prob_pos,
        marker_color=colors_prob,
        text=[f'{p:.0f}%' for p in prob_pos],
        textposition='outside',
        showlegend=False
    ), row=1, col=3)
    fig.add_hline(y=50, line_dash="dash", line_color="gray", row=1, col=3)
    
    fig.update_layout(
        title=dict(
            text=f'<b>⚠️ {country} Stress Test Scenarios (1-Year)</b><br>'
                 '<sup>Performance under different market conditions</sup>',
            x=0.5, font=dict(size=16)
        ),
        height=450, width=1300,
        template='plotly_white'
    )
    
    fig.update_xaxes(tickangle=45)
    
    return fig


if 'SPY' in stress_results:
    fig_stress = create_stress_test_chart(stress_results['SPY'], 'SPY')
    fig_stress.show()

---
# 📉 SECTION E: Value at Risk Comparison

In [21]:
# VaR Comparison Across Countries
def create_var_comparison(returns: pd.DataFrame) -> go.Figure:
    """
    Compare VaR across all countries.
    """
    data = []
    for country in returns.columns:
        if country == 'SPY':
            continue
        ret = returns[country].dropna()
        if len(ret) > 50:
            var_95 = np.percentile(ret, 5)
            data.append({
                'Country': country,
                'Region': COUNTRY_TO_REGION.get(country, 'Unknown'),
                'VaR 95%': var_95,
                'CVaR 95%': ret[ret <= var_95].mean(),
                'Max Loss': ret.min(),
            })
    
    df = pd.DataFrame(data).sort_values('VaR 95%')
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        y=df['Country'], x=df['VaR 95%'],
        name='VaR 95%', orientation='h',
        marker_color='#ff7f0e'
    ))
    fig.add_trace(go.Bar(
        y=df['Country'], x=df['CVaR 95%'],
        name='CVaR 95%', orientation='h',
        marker_color='#d62728'
    ))
    fig.add_trace(go.Bar(
        y=df['Country'], x=df['Max Loss'],
        name='Max Weekly Loss', orientation='h',
        marker_color='#7f7f7f'
    ))
    
    fig.update_layout(
        title=dict(
            text='<b>📉 Value at Risk Comparison (Weekly)</b><br>'
                 '<sup>VaR (95%), CVaR (Expected Shortfall), Maximum Weekly Loss</sup>',
            x=0.5, font=dict(size=16)
        ),
        barmode='group',
        xaxis_title='Return (%)',
        height=600, width=1000,
        template='plotly_white',
        legend=dict(orientation='h', y=1.02)
    )
    
    return fig


fig_var = create_var_comparison(weekly_returns)
fig_var.show()

---
# 💾 SECTION F: Export Results

In [22]:
# Save all results
output_dir = './output_v4'
os.makedirs(output_dir, exist_ok=True)

# Statistics CSV
stats_df.to_csv(f'{output_dir}/country_statistics.csv', index=False)
print(f"✅ Saved: {output_dir}/country_statistics.csv")

# Monte Carlo summary
mc_df = pd.DataFrame(mc_summary)
mc_df.to_csv(f'{output_dir}/monte_carlo_summary.csv', index=False)
print(f"✅ Saved: {output_dir}/monte_carlo_summary.csv")

# Weekly returns
weekly_returns.to_csv(f'{output_dir}/weekly_returns.csv')
print(f"✅ Saved: {output_dir}/weekly_returns.csv")

# Factor exposures
if factor_exposures:
    factor_data = []
    for country, exp in factor_exposures.items():
        row = {'Country': country, 'Alpha': exp['alpha'], 'R2': exp['r_squared']}
        row.update(exp['betas'])
        factor_data.append(row)
    pd.DataFrame(factor_data).to_csv(f'{output_dir}/factor_exposures.csv', index=False)
    print(f"✅ Saved: {output_dir}/factor_exposures.csv")

print(f"\n📁 All files saved to: {os.path.abspath(output_dir)}")

✅ Saved: ./output_v4/country_statistics.csv
✅ Saved: ./output_v4/monte_carlo_summary.csv
✅ Saved: ./output_v4/weekly_returns.csv
✅ Saved: ./output_v4/factor_exposures.csv

📁 All files saved to: c:\Users\Ferhat\Documents\GitHub\Stocks\output_v4


---
# 🎯 Summary

This notebook provides institutional-grade analysis including:

1. **Options-Implied Volatility** - VIX term structure, IV-RV spread, volatility risk premium
2. **Correlation Regimes** - How correlations change between calm and crisis periods
3. **Factor Decomposition** - Exposures to Market, Value, Momentum, Quality, Size, Low Vol
4. **Monte Carlo Simulations** - 10,000 path simulations with VaR, CVaR, and stress testing
5. **60+ Geopolitical Events** - Trump tariffs, Israel-Iran, Ukraine-Russia, semiconductors

**Key Takeaways:**
- Correlations increase during high volatility (diversification fails when needed most)
- VIX typically trades above realized volatility (volatility risk premium)
- Different countries have different factor exposures
- Monte Carlo shows probability distributions for forward scenarios

In [23]:
print("\n" + "=" * 70)
print("🎉 ANALYSIS COMPLETE!")
print("=" * 70)
print(f"\n📊 Analyzed {len(prices.columns)} international indices")
print(f"📅 Date range: {prices.index.min().strftime('%Y-%m-%d')} to {prices.index.max().strftime('%Y-%m-%d')}")
print(f"🎲 Monte Carlo simulations: {len(mc_results)} markets")
print(f"🎯 Factor analysis: {len(factor_exposures)} countries")
print(f"📰 Geopolitical events: {len(GEOPOLITICAL_EVENTS)}")


🎉 ANALYSIS COMPLETE!

📊 Analyzed 16 international indices
📅 Date range: 2024-01-23 to 2026-01-23
🎲 Monte Carlo simulations: 6 markets
🎯 Factor analysis: 15 countries
📰 Geopolitical events: 39
